<a href="https://colab.research.google.com/github/willismax/MediaSystem-Python-Course/blob/main/04.Selenium/%E7%B6%B2%E9%A0%81%E6%93%B7%E5%8F%96_Selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 網頁擷取_Selenium

![image.png](https://hackmd.io/_uploads/H1hC-R1ma.png)



- 官方網站介紹
    - [Selenium](https://www.selenium.dev/)
    - [下載Driver](https://www.selenium.dev/selenium/docs/api/py/index.html) (4.X版已經不用額外安裝)


選擇元素的各種方法
```python
find_element(By.ID, "fruits")
find_element(By.NAME, "q")
find_element(By.CLASS_NAME, "tomatoes")
find_elements(By.TAG_NAME, "li")          #html標籤
find_element(By.LINK_TEXT, "More information...")
find_element(By.PARTIAL_LINK_TEXT, )
find_element(By.XPATH, )           #最常用    #/html開頭為絕對路徑；//相對路徑
find_element(By.CSS_SELECTOR,"#fruits .tomatoes")

```


In [1]:
#下載安裝相關套件，以下為Colab的安裝方式

%pip install -U selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00


### 常用的`webdriver.ChromeOptions()`參數
- start-maximized: Chrome開啟最大化視窗
- incognito: 無痕模式開啟
- headless: 不開啟Chrome模式
- disable-extensions: 停用Chrome擴充功能
- disable-popup-blocking: 停用Chrome彈出視窗
- make-default-browser: 設為預設瀏覽器
- version: 顯示瀏覽器版本
- disable-infobars: 阻止 Chrome 顯示通知“Chrome 由自動化軟體控制”

In [2]:
#@title 以 Selenium 爬取 PTT 指定目標

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


options = Options()
options.add_argument("--headless")
options.add_argument('--no-sandbox') #在背景執行
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--start-maximized")



wd = Chrome(options=options)
url = "https://www.ptt.cc/bbs/Gossiping/M.1689707284.A.6C7.html"
wd.get(url)

while (wd.current_url=='https://www.ptt.cc/ask/over18?from=%2Fbbs%2FGossiping%2FM.1689707284.A.6C7.html'):  #跳轉詢問18歲詢問頁面時，注意URL?from的變化
  wd.find_element(By.CLASS_NAME, "over18-button-container").click()

# 改XPATH定位
# 改SELECTOR定位


elem = wd.find_element(By.LINK_TEXT,'https://hackmd.io/_uploads/S1R4G8N52.jpg')
print(elem.text)
# wd.close()


https://hackmd.io/_uploads/S1R4G8N52.jpg




---



### 以 Selenium 搜尋 PTT 指定列表


![image](https://hackmd.io/_uploads/SyBjKUY4T.png)

In [8]:
#@title 以 Selenium 搜尋 PTT 指定列表

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = Chrome(options=options)
url ='https://www.ptt.cc/bbs/movie/index.html'
wd.get(url)

elem = wd.find_element(By.CLASS_NAME,"title")
print(elem.text)

#beautifulsoup解析
soup = BeautifulSoup(wd.page_source,"html.parser")
links = soup.select('div.title > a')

# title = [ link.get_text() for link in links if '板規' not in link.get_text()]
# href = [ f'https://www.ptt.cc{link.get("href")}' for link in links ]

res = [
    {'title': link.get_text(),
     'href': f'https://www.ptt.cc{link.get("href")}'
     } for link in links if '公告' not in link.get_text() ]


print(res)
wd.close()

[討論] 異形:聖約 一個小問題
[{'title': '[討論] 異形:聖約  一個小問題', 'href': 'https://www.ptt.cc/bbs/movie/M.1711856856.A.531.html'}, {'title': '[討論] 最近看了三部電影的一點想法', 'href': 'https://www.ptt.cc/bbs/movie/M.1711857618.A.C36.html'}]


In [6]:
import pandas as pd

pd.DataFrame(res)

,title,href
0,[討論] 異形:聖約 一個小問題,https://www.ptt.cc/bbs/movie/M.1711856856.A.53...
1,[討論] 最近看了三部電影的一點想法,https://www.ptt.cc/bbs/movie/M.1711857618.A.C3...


In [7]:
pd.DataFrame(res)['href']

0    https://www.ptt.cc/bbs/movie/M.1711856856.A.53...
1    https://www.ptt.cc/bbs/movie/M.1711857618.A.C3...
Name: href, dtype: object



---



In [9]:
#@title Yahoo搜尋
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from bs4 import BeautifulSoup


options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome(options=options)
url='https://tw.yahoo.com/'
wd.get(url)

# 操作網頁元素
element = wd.find_element(By.ID, 'header-search-input')
key_word = 'OpenAI' #@param
element.send_keys(key_word)
wd.find_element(By.ID, 'header-desktop-search-button').click()

# 等待目標表格'id 為 web'的div出現
WebDriverWait(wd, 5).until(
    expected_conditions.presence_of_element_located((By.ID, 'web')))

#然後就是beautifulsoup的範疇了，將目前頁面用bs4解析
soup = BeautifulSoup(wd.page_source,"html.parser")
links = soup.select('div#web h3')

for link in links:
    print(link.get_text())

wd.quit()

openai.comOpenAI
chat.openai.comChatGPT
openai.com › chatgptChatGPT - OpenAI
openai.com › product › gpt-4GPT-4 - OpenAI
OpenAI API
Product - OpenAI
About - OpenAI
openai.com › blog › chatgptIntroducing ChatGPT - OpenAI
zh.wikipedia.org › zh-tw › OpenAIOpenAI - 維基百科，自由的百科全書


In [ ]:
#@title Google搜尋
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from bs4 import BeautifulSoup


options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome(options=options)
url = 'https://www.google.com/'
wd.get(url)

# 用XPATH定位元素 - 搜尋框
element = wd.find_element(By.XPATH,"/html/body/div[4]/div[2]/form/div[1]/div[1]/div[2]/div/div[2]/textarea")
key_word = '\u9032\u64CA\u7684\u5DE8\u4EBA'
element.send_keys(key_word)

# 用XPATH定位元素 - 點選搜尋
wd.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[4]/center/input[1]').click()


# 用BeautifulSoup解析 - 標題h3
soup = BeautifulSoup(wd.page_source,"html.parser")
links = soup.select('div > a > h3')

for link in links:
    print(link.get_text())

wd.quit()

In [ ]:
#@title PChome 搜尋 PS5
# https://blog.jiatool.com/posts/pchome_spider01/
# 實際搶單可參考: https://github.com/ywchiu/largitdata/blob/master/code/Course_137.ipynb
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


options = Options()
options.add_argument("--headless")
options.add_argument('--no-sandbox') #在背景執行
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--start-maximized")
options.add_argument('--disable-dev-shm-usage')

wd = Chrome(options=options)

key_word = 'ps5'

url = f'https://ecshweb.pchome.com.tw/search/v3.3/?q={key_word}'
wd.get(url)

WebDriverWait(wd, 5).until(
    expected_conditions.presence_of_element_located((By.TAG_NAME, 'h5')))

#beautifulsoup解析
soup = BeautifulSoup(wd.page_source,"html.parser")
# wd.quit()

elems = soup.select('h5 > a')

items = [{
    'title': elem.get_text(),
    'link': f'https{elem.get("href")}'}
    for elem in elems]


items

In [ ]:
#@title MOMO搶PS5
# https://ithelp.ithome.com.tw/articles/10262268
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


options = Options()
options.add_argument("--headless")
options.add_argument('--no-sandbox') #在背景執行
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--start-maximized")

prefs = {'profile.default_content_setting_values':{'notifications': 2}}
options.add_experimental_option('prefs', prefs)

driver = Chrome(options=options)

driver.get("https://m.momoshop.com.tw/mymomo/login.momo") # 到登入頁面

driver.find_element(By.ID,'memId').send_keys('帳號') # 輸入帳號
driver.find_element(By.ID,'passwd').send_keys('密碼') # 輸入密碼
driver.find_element(By.CLASS_NAME,'login').click()
driver.get("https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=9435324")

count=0
while count<2:
  try:
    buy = WebDriverWait(driver, 1, 0.5).until(EC.presence_of_element_located((By.ID, 'buy_yes'))) # 顯性等待
    buy.click() # 偵測到可以購買按鈕就點擊按鈕
    print ('可以購買!')
    break # 後面結帳部分就不寫囉
  except:
    print("還不能購買! 重新整理!")
    driver.refresh() # 重整頁面
    count+=1